In [1]:
import os
import re
import pandas as pd
import numpy as np
from tensorflow import keras
import tensorflow as tf

C:\Users\91999\anaconda3\envs\myenv2\lib\site-packages\tensorflow\python\keras\engine\training_arrays_v1.py:37: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.21.6)
  from scipy.sparse import issparse  # pylint: disable=g-import-not-at-top


In [2]:
model_path = r'C:\Users\91999\OneDrive\Desktop\DRR project\general\trained_model\A1\DRR_MTL_3c_1R_250e'

In [3]:
base_path = r'D:\new camera'

In [4]:
output_path = r'C:\Users\91999\OneDrive\Desktop\trial A1.xlsx'

In [5]:
# Define image dimensions and load the model
img_dim = 224

model =  tf.keras.models.load_model(model_path)

In [6]:
# Extract image file paths
image_file_paths = []
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith((".png", ".PNG", ".jpeg")):  # Add other image extensions if necessary
            image_file_paths.append(os.path.join(root, file))


In [7]:
# Load test dataset without shuffling to maintain order
test_ds = keras.utils.image_dataset_from_directory(
    base_path,
    image_size=(img_dim, img_dim),
    label_mode='int',
    batch_size=1000,
    shuffle=False  # Ensure no shuffling to keep image file order intact
)

Found 846 files belonging to 2 classes.


In [8]:
class_names = test_ds.class_names

In [9]:
resize_and_rescale = keras.Sequential([
  #layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  keras.layers.experimental.preprocessing.Rescaling(1./255)
])

In [10]:
data_augmentation = keras.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    keras.layers.experimental.preprocessing.RandomRotation(0.1),
    #keras.layers.experimental.preprocessing.RandomFlip("vertical"),
])

In [11]:
# Function to prepare dataset
AUTOTUNE = tf.data.AUTOTUNE
def prepare(ds, augment=False):
    def resize_and_rescale(image, label):
        image = tf.image.resize(image, [img_dim, img_dim])
        image = tf.cast(image, tf.float32) / 255.0
        return image, label
    
    # Apply resizing and rescaling
    ds = ds.map(lambda x, y: (resize_and_rescale(x, y)), num_parallel_calls=AUTOTUNE)
    
    # Apply data augmentation if specified
    if augment:
        data_augmentation = tf.keras.Sequential([
            tf.keras.layers.RandomFlip("horizontal_and_vertical"),
            tf.keras.layers.RandomRotation(0.2)
        ])
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)
        
    # Use buffered prefetching
    return ds.prefetch(buffer_size=AUTOTUNE)


In [12]:
# Prepare the test dataset
test_ds = prepare(test_ds)


In [13]:
# Separate the images and labels
ts_image_batch,tr_label_batch = next(iter(test_ds))
ts_image_batch = np.asarray(ts_image_batch)

In [14]:
preds = model.predict(ts_image_batch)


27/27 [==============================] - 118s 4s/step


In [15]:
# Extract predicted class indices and scores
y_pred_3c = []
for i in preds[0]:
    y_pred_3c = np.append(y_pred_3c,np.argmax(i))

y_pred_scores = []
for i in preds[1]:
    y_pred_scores = np.append(y_pred_scores,int(abs(np.round(i,0))))

In [16]:
actual_classes = [class_names[label] for label in tr_label_batch]
L = ['CONTROL','PATHOGEN/NON DRR','PATHOGEN/NON DRR']
y_pred_classes = [L[int(label)] for label in y_pred_3c]

In [17]:
# Extract image file names using regex
#image_file_names = [re.match(file_name_pattern, file_path).group(1) for file_path in image_file_paths[:len(ts_label_batch)]]
image_file_names = [os.path.basename(file_path) for file_path in image_file_paths[:len(ts_image_batch)]]

In [18]:
print(len(image_file_names), len(y_pred_classes), len(y_pred_scores))

846 846 846


In [19]:
image_file_paths = []
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith((".png", ".jpg", ".jpeg")):  # Add other image extensions if necessary
            image_file_paths.append(os.path.join(root, file))

print("Total images found:", len(image_file_paths))


Total images found: 846


In [20]:
# Create a DataFrame with Image Name, Predicted Class, Predicted Score, Actual Class
results_df = pd.DataFrame({
    'Folder': actual_classes,
    'Image Name': image_file_names,
    'Predicted Class': y_pred_classes,
    'Predicted Score': y_pred_scores
})


In [21]:
# Save the DataFrame to an Excel file with a different name

results_df.to_excel(excel_writer=output_path, index=False)

print(f"Results have been saved to {output_path}")


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 2)
display(results_df)

Results have been saved to C:\Users\91999\OneDrive\Desktop\trial A1.xlsx


,Folder,Image Name,Predicted Class,Predicted Score
0,CONTROL,DSCF5264.png,NON DRR,6.0
1,CONTROL,DSCF5265.png,NON DRR,6.0
2,CONTROL,DSCF5266.png,NON DRR,6.0
3,CONTROL,DSCF5267.png,NON DRR,6.0
4,CONTROL,DSCF5268.png,NON DRR,6.0
5,CONTROL,DSCF5269.png,NON DRR,6.0
6,CONTROL,DSCF5270.png,NON DRR,6.0
7,CONTROL,DSCF5271.png,NON DRR,6.0
8,CONTROL,DSCF5272.png,NON DRR,5.0
9,CONTROL,DSCF5273.png,NON DRR,6.0
